# Signal Classification with BDTs

Consider a search for a Vector-like quark (VLQ) particle of unkown mass. 

The search is performed in the $B\to\;b\;H(\to\;\gamma\gamma)$ channel.

Simulation samples have been generated for different VLQ masses: 800, 1000, 1200, 1500, 1800, 2000, 2200 GeV

This notebook explores the performance of a BDT (trained using a single VLQ mass) in separating signal candidates from $tHq$ candidates (dominant bkg).

### Load datasets

Load the signal trees for different VLQ mass points

In [ ]:
import uproot

masses = [800, 1000, 1200, 1500, 1800, 2000, 2200]
datadir = "../data"
treename = "BbH_tree"
trees = {
    mass: uproot.open(f"{datadir}/BDT_tree_M{mass}_14TeV.root")[treename]
    for mass in masses
}

Load the background tree

In [ ]:
trees[0] = uproot.open(f"{datadir}/BKG_tree_tHq_14TeV.root")[treename]

### Import variable definitions and groups of variables

In [ ]:
from b2bH_vlq import get_variable_group_names, get_variables_by_group
get_variable_group_names()

In [ ]:
all_vars = {}
for gr in get_variable_group_names():
    all_vars.update(get_variables_by_group(gr))
all_vars.keys()

### Vizualize distributions

#### Plot reconstructed $m(b\gamma\gamma)$ distributions.

In [ ]:
from hepkit.histograms import hist1d_from_var

def fill_histograms(group:str):
    vars = get_variables_by_group(group)
    hists ={}
    for mass in masses + [0]:
        tree = trees[mass]
        hists[mass] = {
            k: hist1d_from_var(
                var,
                tree,
            )
            for k, var in vars.items()
        }
    return hists

hist_masses = fill_histograms("masses") 

In [ ]:
import matplotlib.pyplot as plt
from hepkit.histograms import plot_hist1d_comparison
from hepkit.plotting import set_cms_style, get_color_palette
set_cms_style(grid=True)

fi, ax = plt.subplots(figsize=(7,6))
colors = get_color_palette(n_colors=len(masses) + 1)
plot_hist1d_comparison(
    [ hist_masses[mass]['VLQ_mass'] for mass in masses + [0] ],
    [ f"Signal M={mass} GeV" for mass in masses ] + ["Background"],
    ax=ax,
    colors=colors,
)

In [ ]:
hist_photon1 = fill_histograms("photon1")
hist_photon2 = fill_histograms("photon2")
hist_diphoton = fill_histograms("diphoton")
hist_bjet1 = fill_histograms("bjet1")
hists_event = fill_histograms("event")

In [ ]:
from hepkit.histograms import multi_hist1d_comparison

colors = get_color_palette(n_colors=len(masses) + 1)
legends = [ f"Signal M={mass} GeV" for mass in masses ] + ["Background"]
histtypes = ["step"] * len(masses) + ["fill"]
for hists in [hist_photon1, hist_photon2, hist_diphoton, hist_bjet1, hists_event]:
    multi_hist1d_comparison(
        [ hists[mass] for mass in masses + [0] ],
        legends,
        histtypes,
        colors,
        figsize_per_plot=(3.0,2.5),
        max_cols=4,
        subplot_titles=False
    );

## Training

#### Train a Catboost classifier using the 800 GeV signal sample.

In [ ]:
sigdf = trees[800].arrays(library="pd")
bkgdf = trees[0].arrays(library="pd")
sigdf.shape, bkgdf.shape

In [ ]:
w_sig = sigdf["evt_weight"]
w_bkg = bkgdf["evt_weight"]

#### Input features

Select as input features kinematic variables $p_{T}$ and $\eta$ from  the VLQ candidate and its decay products.

Also include $H_{T}$ and deltaR between b-jet and Higgs candidates.

Include multiplicity (n-b-jets, n-fwd-jets) as categoraical features.

In [ ]:
from itertools import product

cands = ["photon1", "photon2", "diphoton", "bjet", "VLQ"]
obs = ["pt", "eta", "phi"]

mva_vars = { }
for cand, obs in product(cands, obs):
    key = f"{cand}_{obs}"
    mva_vars[key] = all_vars[key]
mva_vars["HT"] = all_vars["HT"]
mva_vars["deltaR_bjet_Higgs"] = all_vars["deltaR_bjet_Higgs"]
mva_vars["bjet_multiplicity"] = all_vars["bjet_multiplicity"]
mva_vars["jet_multiplicity"] = all_vars["jet_multiplicity"]
mva_vars["forwardjet_multiplicity"] = all_vars["forwardjet_multiplicity"]
mva_vars.keys();

#### Compare signal and background distributions

In [ ]:
mva_names = list(mva_vars.keys())
hist_mva_sig = {}
hist_mva_bkg = {}
for name in mva_names:
    var = mva_vars[name]
    hist_mva_sig[name] = hist1d_from_var(var, sigdf)
    hist_mva_bkg[name] = hist1d_from_var(var, bkgdf)

In [ ]:
from hepkit.classification.visualization import plot_signal_background_comparison
fig = plot_signal_background_comparison(
    hist_mva_sig, hist_mva_bkg, subplot_titles=False, max_cols=4, figsize_per_plot=(2.5,2.),
)
plt.tight_layout()

Preprocess the data and prepare training and validation samples

In [ ]:
from sklearn.model_selection import train_test_split
from hepkit.classification.preprocessing import prepare_training_data

In [ ]:
Xy = prepare_training_data(
    sigdf,
    bkgdf,
    mva_vars.values(),
    mva_vars.values(),
    #sig_weights=w_sig,
    #bkg_weights=w_bkg,
    # id_columns=["NEvts"],
    cat_vars=["bjet_multiplicity", "jet_multiplicity", "forwardjet_multiplicity"]
)

In [ ]:
# Define the training and validation sets
train_X, test_X, train_y, test_y = train_test_split(
    Xy.drop("label", axis=1), Xy["label"], test_size=0.15, random_state=42, stratify=Xy["label"]
)
#train_X, val_X, train_y, val_y = train_test_split(
#    train_X, train_y, test_size=0.15, random_state=42, stratify=train_y
#)

In [ ]:
# Get the ratio of signal and background events in the training set and validation set
n_sig_train = sum(train_y==1)
n_bkg_train = sum(train_y==0)

sigfrac_train = n_sig_train / (n_sig_train + n_bkg_train)
sigfrac_train, n_sig_train / n_bkg_train

In [ ]:
#train_weights = train_X.pop("weights")
#val_weights = val_X.pop("weights")

### CatBoost classifier

In [ ]:
import catboost
from catboost import CatBoostClassifier, Pool, EFeaturesSelectionAlgorithm, EShapCalcType

In [ ]:
cat_features = ["bjet_multiplicity", "jet_multiplicity", "forwardjet_multiplicity"]
train_pool = Pool(data=train_X, label=train_y, cat_features=cat_features)
# val_pool = Pool(data=val_X, label=val_y, cat_features=cat_features)

# train_pool = Pool(data=train_X, label=train_y, weight=train_weights)
# val_pool = Pool(data=val_X, label=val_y, weight=val_weights)

In [ ]:
train_pool.get_embedding_feature_indices()


Train with almost default parameters and small learning rate.

Use CatBoost's `cv` function to check the number of iterations

In [ ]:
weight_bkg = n_sig_train / n_bkg_train

cb_params = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 5000,
    "random_state": 42,
    "learning_rate": 0.01,
    #"depth": 4,
    #"class_weights": [weight_bkg, 1.0],
    #'rsm': 0.5,
    #'reg_lambda': 1,
    #"custom_metric": ["Accuracy", "Precision"],  # "Recall", "F1"],
    #"od_wait": 100,
}

In [ ]:
import math
import mlflow
import numpy as np

from hepkit.classification.visualization import plot_roc_auc, plot_train_test_response

from sklearn.model_selection import StratifiedKFold

def run_experiment(
    train_pool,
    test_pool=None,
    cv_folds=3,
    plot=False,
    verbose=False,
    random_seed=42,
    **params,
):
    defaults = {
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "iterations": 3000,
        "learning_rate": 0.01,
        "random_state": random_seed,
        "od_wait": 100,
    }

    model_params = defaults | params
    od_wait = model_params.pop("od_wait", 100)
    
    splitter = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_seed)

    # run cross-validation
    cv_data, cv_models = catboost.cv(
        params=model_params,
        pool=train_pool,
        partition_random_seed=random_seed,
        folds = splitter,
        verbose=verbose,
        plot=plot,
        return_models=True,
    )

    # find best iteration
    best_iter_auc = cv_data["test-AUC-mean"].values.argmax()
    best_auc = cv_data["test-AUC-mean"].values[best_iter_auc]
    best_auc_std = cv_data["test-AUC-std"].values[best_iter_auc]

    best_iter_loss = cv_data["test-Logloss-mean"].values.argmin()
    best_loss = cv_data["test-Logloss-mean"].values[best_iter_loss]
    best_loss_std = cv_data["test-Logloss-std"].values[best_iter_loss]

    decimals = max(0, math.ceil(-math.log10(best_auc_std))) if best_auc_std > 0 else 0
    print(f"Best validation AUC: {best_auc:.{decimals}f} +/- {best_auc_std:<.{decimals}f} at iteration {best_iter_auc}")
    
    decimals = max(0, math.ceil(-math.log10(best_loss_std))) if best_loss_std > 0 else 0
    print(f"Best validation Logloss: {best_loss:.{decimals}f} +/- {best_loss_std:<.{decimals}f} at iteration {best_iter_loss}")
    
    y = train_pool.get_label()
    X_dummy = np.zeros(len(y))
    splits = splitter.split(X_dummy, y)
    train_scores, val_scores = [], []
    train_y, val_y = [], []
    for i, (train_idx, val_idx) in enumerate(splits):
        train_fold = train_pool.slice(train_idx)
        val_fold = train_pool.slice(val_idx)
        train_proba = cv_models[i].predict(
            train_fold, 
            prediction_type='Probability', 
            ntree_end=int(best_iter_auc)
        )[:, 1]
        
        val_proba = cv_models[i].predict(
            val_fold, 
            prediction_type='Probability', 
            ntree_end=int(best_iter_auc)
        )[:, 1]
        train_scores.append(train_proba)
        val_scores.append(val_proba)
        train_y.append(train_fold.get_label())
        val_y.append(val_fold.get_label())
        
    fig = plot_roc_auc(
        train_y + val_y, 
        train_scores + val_scores,
        labels = [f"train fold {i}" for i in range(cv_folds)] + [f"validation fold {i}" for i in range(cv_folds)], 
        fig_size=(6,4),
        style="rejection"
    )

    model_params["iterations"] = best_iter_auc + od_wait
    model_params["custom_metric"] = ["AUC:hints=skip_train~false"]
    model_params["od_wait"] = od_wait
    model = CatBoostClassifier(**model_params)

    model.fit(train_pool, plot=plot, verbose=verbose)
    params = model.get_params()

    if mlflow.active_run():
        mlflow.log_params(model_params)
        mlflow.log_metric("cv_auc", best_auc)
        mlflow.log_metric("cv_logloss", best_loss)
        mlflow.log_metric("train_auc", model.get_best_score()["learn"]["AUC"])
        mlflow.log_metric("train_logloss", model.get_best_score()["learn"]["Logloss"])
        mlflow.log_figure(fig, "plots/cv_roc_auc.png")
        plt.close(fig)

    return model

In [ ]:
model = run_experiment(train_pool)

In [ ]:
import os
from dotenv import load_dotenv
from mlflow import MlflowClient

# Load environment variables from .env file
load_dotenv()

# Verify environment variables are loaded
print(f"MLFLOW_TRACKING_URI: {os.getenv('MLFLOW_TRACKING_URI')}")
print(f"MLFLOW_TRACKING_TOKEN: {'*' * 20 if os.getenv('MLFLOW_TRACKING_TOKEN') else 'Not set'}")

In [ ]:
client = MlflowClient()

In [ ]:
try:
    experiment_id = mlflow.create_experiment(name="catboost-M800-tHq")
except mlflow.exceptions.RestException:
    # Experiment already exists, get its ID
    experiment = mlflow.get_experiment_by_name("catboost-M800-tHq")
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_id=experiment_id)

In [ ]:
with mlflow.start_run(run_name="initial"):
    model = run_experiment(train_pool)

PLot learning curve

In [ ]:
from hepkit.classification.visualization import plot_learning_curve
fig = plot_learning_curve(model, metric="Logloss", fig_size=(6,4))

In [ ]:
from hepkit.classification.visualization import plot_roc_auc
fig = plot_roc_auc(val_y, val_proba[:, 1], fig_size=(6,4), style="rejection")

In [ ]:
import numpy as np
def plot_feature_importance(
    model: CatBoostClassifier,
    feature_names,
    max_num_features: int = 20,
    ax: plt.Axes | None = None,
):
    """
    Plot the feature importance of a CatBoost model and return the axes used.
    """
    import matplotlib.pyplot as plt

    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))

    feature_importances = model.feature_importances_
    indices = np.argsort(feature_importances)[-max_num_features:]
    ax.barh(
        range(len(indices)), feature_importances[indices], color="b", align="center"
    )
    ax.set_yticks(range(len(indices)))
    ax.set_yticklabels([feature_names[i] for i in indices])
    ax.set_xlabel("Relative Importance")
    ax.set_title("Feature Importances")
    return ax


In [ ]:
fig,ax = plt.subplots(figsize=(8,6));
plot_feature_importance(model, train_X.columns, max_num_features=20, ax=ax);

## xGboost

In [ ]:
import xgboost as xgb

In [ ]:
model_xgb = xgb.XGBClassifier(
    n_estimators=2500,
    #max_depth=2,
    learning_rate=0.01,
    random_state=42,
    early_stopping_rounds=100,
)

In [ ]:
model_xgb.fit(train_X, train_y, eval_set=[(val_X, val_y)], verbose=False);

In [ ]:
train_predict_xgb = model_xgb.predict(train_X)
train_proba_xgb = model_xgb.predict_proba(train_X) # these are the scores
val_predict_xgb = model_xgb.predict(val_X)
val_proba_xgb = model_xgb.predict_proba(val_X) # these are the scores

In [ ]:
plot_train_test_response(
    model_xgb,
    train_X, train_y, val_X, val_y, log_y=False
)
plot_train_test_response(
    model_xgb,
    train_X, train_y, val_X, val_y, log_y=True
)


In [ ]:
plot_signal_efficiency_vs_background_rejection(train_y, train_proba_xgb[:, 1])